In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LetNet5
import tf_general as tfg
import numpy as np


FLAGS = tf.flags.FLAGS
try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 10000, 'epoch')
    tf.flags.DEFINE_integer('batch_size', 512, 'batch size')
    tf.flags.DEFINE_integer('test_size', 5000, 'test size')
    tf.flags.DEFINE_float('lr', 0.01, 'learning rate')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

CONTINUE = 1

parameters were defined.


In [2]:
data = cifar10();
train_image, train_label = data.get_train_batch(FLAGS.batch_size)
#test_image, test_label = data.get_test_batch(FLAGS.test_size)

train set length: 50000
test image shape: (10000, 32, 32, 3)
test label shape: (10000,)
test set length: 10000


In [3]:
ckpt_dir = 'ckpt/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 28,28,3], name='x_input')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    y_ = tf.placeholder(tf.int64, [None], name='labels')

In [5]:
with tf.name_scope('prediction'):
    le_net5 = LetNet5(x, keep_prob)
    fc2 = le_net5.fc2
    logits = le_net5.logits
    y = le_net5.prediction

Instructions for updating:
dim is deprecated, use axis instead


In [6]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                  labels=y_, 
                                                                                  name="cross_entropy_per_example"))

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [8]:
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
import time

if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep = 1)
        if CONTINUE != 0:
            model_file=tf.train.latest_checkpoint('ckpt/')
            saver.restore(sess,model_file)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)
        for i in range(FLAGS.epoch):
            train_image, train_label = data.get_train_batch(FLAGS.batch_size)
            #print(len(train_image))
            #print(len(train_label))
            _,accuracy_rate = sess.run([train_step,accuracy], feed_dict={keep_prob: 0.5, x:train_image, y_:train_label})
            log = []
            string =time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter:' + str(i)+', accuracy='+str(round(accuracy_rate*100,2))+'%'
            log.append(string)
            print(string)
            tfg.saveEvalData('lenet5_cifar.log',log)            
            if (i+1) % 250 == 0:  #保存预测模型
                saver.save(sess,'ckpt/cifar10_'+str(i+1)+'.ckpt',global_step=i)  
                for j in range(2):                    
                    test_image, test_label = data.get_test_batch(FLAGS.test_size)
                    accuracy_rate += sess.run(accuracy,feed_dict={keep_prob: 1, x:test_image, y_:test_label})
                accuracy_rate = accuracy_rate/2
                log = []
                string = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) + ' iter ' + str(i) + ', Test accuracy:' +str(round(accuracy_rate*100,2))+'%'
                print(string)
                log.append(string)
                tfg.saveEvalData('lenet5_cifar.log',log)
    tf.reset_default_graph()

INFO:tensorflow:Restoring parameters from ckpt/cifar10_5000.ckpt-4999
2019-05-22 13:55:28 iter:0, accuracy=41.6%
2019-05-22 13:55:35 iter:1, accuracy=41.8%
2019-05-22 13:55:43 iter:2, accuracy=40.43%
2019-05-22 13:55:53 iter:3, accuracy=41.41%
2019-05-22 13:56:05 iter:4, accuracy=41.8%
2019-05-22 13:56:17 iter:5, accuracy=41.99%
2019-05-22 13:56:31 iter:6, accuracy=44.14%
2019-05-22 13:56:45 iter:7, accuracy=40.23%
2019-05-22 13:56:55 iter:8, accuracy=39.65%
2019-05-22 13:57:03 iter:9, accuracy=40.04%
2019-05-22 13:57:11 iter:10, accuracy=40.04%
2019-05-22 13:57:21 iter:11, accuracy=39.06%
2019-05-22 13:57:29 iter:12, accuracy=35.16%
2019-05-22 13:57:37 iter:13, accuracy=38.48%
2019-05-22 13:57:45 iter:14, accuracy=40.62%
2019-05-22 13:57:54 iter:15, accuracy=42.58%
2019-05-22 13:58:02 iter:16, accuracy=41.41%
2019-05-22 13:58:11 iter:17, accuracy=44.73%
2019-05-22 13:58:19 iter:18, accuracy=36.33%
2019-05-22 13:58:29 iter:19, accuracy=43.55%
2019-05-22 13:58:37 iter:20, accuracy=37.7%

2019-05-22 14:25:19 iter:180, accuracy=42.58%
2019-05-22 14:25:29 iter:181, accuracy=36.33%
2019-05-22 14:25:39 iter:182, accuracy=41.02%
2019-05-22 14:25:51 iter:183, accuracy=38.67%
2019-05-22 14:26:02 iter:184, accuracy=37.7%
2019-05-22 14:26:14 iter:185, accuracy=40.62%
2019-05-22 14:26:26 iter:186, accuracy=43.36%
2019-05-22 14:26:39 iter:187, accuracy=41.02%
2019-05-22 14:26:53 iter:188, accuracy=43.95%
2019-05-22 14:27:08 iter:189, accuracy=37.89%
2019-05-22 14:27:26 iter:190, accuracy=39.26%
2019-05-22 14:27:49 iter:191, accuracy=39.26%
2019-05-22 14:28:19 iter:192, accuracy=41.6%
2019-05-22 14:29:37 iter:193, accuracy=37.7%
2019-05-22 14:33:31 iter:194, accuracy=42.58%
2019-05-22 14:33:37 iter:195, accuracy=38.87%
2019-05-22 14:33:42 iter:196, accuracy=39.45%
2019-05-22 14:33:47 iter:197, accuracy=44.34%
2019-05-22 14:33:53 iter:198, accuracy=41.41%
2019-05-22 14:33:58 iter:199, accuracy=35.74%
2019-05-22 14:34:04 iter:200, accuracy=42.19%
2019-05-22 14:34:09 iter:201, accurac

2019-05-22 14:56:51 iter:358, accuracy=41.6%
2019-05-22 14:56:57 iter:359, accuracy=43.16%
2019-05-22 14:57:04 iter:360, accuracy=38.28%
2019-05-22 14:57:10 iter:361, accuracy=38.87%
2019-05-22 14:57:17 iter:362, accuracy=38.87%
2019-05-22 14:57:23 iter:363, accuracy=41.41%
2019-05-22 14:57:30 iter:364, accuracy=37.3%
2019-05-22 14:57:37 iter:365, accuracy=39.84%
2019-05-22 14:57:43 iter:366, accuracy=41.02%
2019-05-22 14:57:50 iter:367, accuracy=39.26%
2019-05-22 14:57:57 iter:368, accuracy=40.43%
2019-05-22 14:58:04 iter:369, accuracy=41.6%
2019-05-22 14:58:12 iter:370, accuracy=43.16%
2019-05-22 14:58:19 iter:371, accuracy=38.67%
2019-05-22 14:58:27 iter:372, accuracy=43.16%
2019-05-22 14:58:34 iter:373, accuracy=41.6%
2019-05-22 14:58:42 iter:374, accuracy=42.38%
2019-05-22 14:58:49 iter:375, accuracy=40.82%
2019-05-22 14:58:57 iter:376, accuracy=38.28%
2019-05-22 14:59:05 iter:377, accuracy=40.82%
2019-05-22 14:59:14 iter:378, accuracy=44.92%
2019-05-22 14:59:24 iter:379, accuracy

2019-05-22 15:26:09 iter:536, accuracy=37.89%
2019-05-22 15:26:14 iter:537, accuracy=47.27%
2019-05-22 15:26:21 iter:538, accuracy=41.6%
2019-05-22 15:26:27 iter:539, accuracy=40.43%
2019-05-22 15:26:32 iter:540, accuracy=38.28%
2019-05-22 15:26:39 iter:541, accuracy=34.57%
2019-05-22 15:26:45 iter:542, accuracy=38.09%
2019-05-22 15:26:51 iter:543, accuracy=39.84%
2019-05-22 15:26:57 iter:544, accuracy=41.8%
2019-05-22 15:27:03 iter:545, accuracy=40.23%
2019-05-22 15:27:09 iter:546, accuracy=39.06%
2019-05-22 15:27:15 iter:547, accuracy=42.58%
2019-05-22 15:27:21 iter:548, accuracy=37.89%
2019-05-22 15:27:28 iter:549, accuracy=43.95%
2019-05-22 15:27:34 iter:550, accuracy=38.87%
2019-05-22 15:27:40 iter:551, accuracy=42.77%
2019-05-22 15:27:47 iter:552, accuracy=37.89%
2019-05-22 15:27:53 iter:553, accuracy=41.41%
2019-05-22 15:28:00 iter:554, accuracy=41.21%
2019-05-22 15:28:06 iter:555, accuracy=41.41%
2019-05-22 15:28:12 iter:556, accuracy=38.48%
2019-05-22 15:28:19 iter:557, accura

2019-05-22 15:54:34 iter:715, accuracy=40.43%
2019-05-22 15:54:40 iter:716, accuracy=41.6%
2019-05-22 15:54:46 iter:717, accuracy=40.04%
2019-05-22 15:54:51 iter:718, accuracy=37.3%
2019-05-22 15:54:57 iter:719, accuracy=43.55%
2019-05-22 15:55:03 iter:720, accuracy=38.67%
2019-05-22 15:55:08 iter:721, accuracy=40.04%
2019-05-22 15:55:14 iter:722, accuracy=44.73%
2019-05-22 15:55:19 iter:723, accuracy=40.43%
2019-05-22 15:55:25 iter:724, accuracy=43.36%
2019-05-22 15:55:31 iter:725, accuracy=43.55%
2019-05-22 15:55:37 iter:726, accuracy=36.13%
2019-05-22 15:55:43 iter:727, accuracy=41.8%
2019-05-22 15:55:49 iter:728, accuracy=40.04%
2019-05-22 15:55:55 iter:729, accuracy=38.67%
2019-05-22 15:56:01 iter:730, accuracy=42.38%
2019-05-22 15:56:07 iter:731, accuracy=40.23%
2019-05-22 15:56:13 iter:732, accuracy=43.16%
2019-05-22 15:56:19 iter:733, accuracy=43.55%
2019-05-22 15:56:24 iter:734, accuracy=38.67%
2019-05-22 15:56:31 iter:735, accuracy=43.75%
2019-05-22 15:56:37 iter:736, accurac

...

2019-05-23 07:53:49 iter:6473, accuracy=45.51%
2019-05-23 07:53:56 iter:6474, accuracy=54.3%
2019-05-23 07:54:01 iter:6475, accuracy=52.34%
2019-05-23 07:54:07 iter:6476, accuracy=51.76%
2019-05-23 07:54:13 iter:6477, accuracy=50.59%
2019-05-23 07:54:18 iter:6478, accuracy=46.48%
2019-05-23 07:54:24 iter:6479, accuracy=51.37%
2019-05-23 07:54:30 iter:6480, accuracy=47.66%
2019-05-23 07:54:35 iter:6481, accuracy=48.44%
2019-05-23 07:54:41 iter:6482, accuracy=49.8%
2019-05-23 07:54:47 iter:6483, accuracy=44.92%
2019-05-23 07:54:52 iter:6484, accuracy=45.9%
2019-05-23 07:54:58 iter:6485, accuracy=46.09%
2019-05-23 07:55:04 iter:6486, accuracy=51.37%
2019-05-23 07:55:09 iter:6487, accuracy=49.41%
2019-05-23 07:55:15 iter:6488, accuracy=49.41%
2019-05-23 07:55:21 iter:6489, accuracy=45.7%
2019-05-23 07:55:27 iter:6490, accuracy=47.07%
2019-05-23 07:55:33 iter:6491, accuracy=51.95%
2019-05-23 07:55:39 iter:6492, accuracy=44.92%
2019-05-23 07:55:45 iter:6493, accuracy=51.37%
2019-05-23 07:55:

2019-05-23 08:22:53 iter:6647, accuracy=47.66%
2019-05-23 08:22:59 iter:6648, accuracy=49.02%
2019-05-23 08:23:04 iter:6649, accuracy=50.59%
2019-05-23 08:23:09 iter:6650, accuracy=48.24%
2019-05-23 08:23:15 iter:6651, accuracy=47.07%
2019-05-23 08:23:20 iter:6652, accuracy=53.91%
2019-05-23 08:23:26 iter:6653, accuracy=48.24%
2019-05-23 08:23:32 iter:6654, accuracy=50.39%
2019-05-23 08:23:38 iter:6655, accuracy=52.15%
2019-05-23 08:23:43 iter:6656, accuracy=49.22%
2019-05-23 08:23:49 iter:6657, accuracy=53.12%
2019-05-23 08:23:55 iter:6658, accuracy=47.66%
2019-05-23 08:24:01 iter:6659, accuracy=49.61%
2019-05-23 08:24:07 iter:6660, accuracy=47.46%
2019-05-23 08:24:13 iter:6661, accuracy=46.88%
2019-05-23 08:24:19 iter:6662, accuracy=50.2%
2019-05-23 08:24:24 iter:6663, accuracy=51.95%
2019-05-23 08:24:31 iter:6664, accuracy=47.85%
2019-05-23 08:24:37 iter:6665, accuracy=50.0%
2019-05-23 08:24:42 iter:6666, accuracy=48.63%
2019-05-23 08:24:48 iter:6667, accuracy=51.76%
2019-05-23 08:2

2019-05-23 08:46:17 iter:6821, accuracy=49.8%
2019-05-23 08:46:26 iter:6822, accuracy=51.17%
2019-05-23 08:46:33 iter:6823, accuracy=47.66%
2019-05-23 08:46:43 iter:6824, accuracy=50.59%
2019-05-23 08:46:52 iter:6825, accuracy=51.95%
2019-05-23 08:47:01 iter:6826, accuracy=50.2%
2019-05-23 08:47:12 iter:6827, accuracy=48.63%
2019-05-23 08:47:22 iter:6828, accuracy=53.12%
2019-05-23 08:47:34 iter:6829, accuracy=51.37%
2019-05-23 08:47:48 iter:6830, accuracy=50.98%
2019-05-23 08:48:03 iter:6831, accuracy=51.95%
2019-05-23 08:48:24 iter:6832, accuracy=49.61%
2019-05-23 08:48:56 iter:6833, accuracy=50.0%
2019-05-23 08:52:53 iter:6834, accuracy=46.29%
2019-05-23 08:52:58 iter:6835, accuracy=52.15%
2019-05-23 08:53:04 iter:6836, accuracy=47.85%
2019-05-23 08:53:09 iter:6837, accuracy=49.41%
2019-05-23 08:53:14 iter:6838, accuracy=50.78%
2019-05-23 08:53:19 iter:6839, accuracy=48.44%
2019-05-23 08:53:25 iter:6840, accuracy=48.44%
2019-05-23 08:53:31 iter:6841, accuracy=51.17%
2019-05-23 08:53

2019-05-23 09:15:12 iter:6996, accuracy=49.41%
2019-05-23 09:15:18 iter:6997, accuracy=53.91%
2019-05-23 09:15:25 iter:6998, accuracy=50.78%
2019-05-23 09:15:32 iter:6999, accuracy=51.37%
2019-05-23 09:16:22 iter 6999, Test accuracy:35.96%
2019-05-23 09:16:28 iter:7000, accuracy=50.98%
2019-05-23 09:16:35 iter:7001, accuracy=52.34%
2019-05-23 09:16:41 iter:7002, accuracy=51.17%
2019-05-23 09:16:47 iter:7003, accuracy=54.88%
2019-05-23 09:16:54 iter:7004, accuracy=47.85%
2019-05-23 09:17:00 iter:7005, accuracy=46.48%
2019-05-23 09:17:07 iter:7006, accuracy=47.27%
2019-05-23 09:17:14 iter:7007, accuracy=47.85%
2019-05-23 09:17:21 iter:7008, accuracy=50.2%
2019-05-23 09:17:28 iter:7009, accuracy=48.83%
2019-05-23 09:17:34 iter:7010, accuracy=51.56%
2019-05-23 09:17:41 iter:7011, accuracy=48.44%
2019-05-23 09:17:48 iter:7012, accuracy=53.12%
2019-05-23 09:17:56 iter:7013, accuracy=47.66%
2019-05-23 09:18:04 iter:7014, accuracy=52.54%
2019-05-23 09:18:12 iter:7015, accuracy=46.09%
2019-05-2

2019-05-23 09:43:27 iter:7170, accuracy=46.88%
2019-05-23 09:43:32 iter:7171, accuracy=48.44%
2019-05-23 09:43:37 iter:7172, accuracy=50.39%
2019-05-23 09:43:43 iter:7173, accuracy=52.34%
2019-05-23 09:43:49 iter:7174, accuracy=47.66%
2019-05-23 09:43:54 iter:7175, accuracy=50.2%
2019-05-23 09:43:59 iter:7176, accuracy=48.63%
2019-05-23 09:44:06 iter:7177, accuracy=50.78%
2019-05-23 09:44:14 iter:7178, accuracy=52.34%
2019-05-23 09:44:23 iter:7179, accuracy=50.59%
2019-05-23 09:44:35 iter:7180, accuracy=47.66%
2019-05-23 09:44:43 iter:7181, accuracy=49.8%
2019-05-23 09:44:51 iter:7182, accuracy=49.61%
2019-05-23 09:44:59 iter:7183, accuracy=48.24%
2019-05-23 09:45:06 iter:7184, accuracy=47.85%
2019-05-23 09:45:13 iter:7185, accuracy=49.02%
2019-05-23 09:45:20 iter:7186, accuracy=50.39%
2019-05-23 09:45:27 iter:7187, accuracy=50.98%
2019-05-23 09:45:33 iter:7188, accuracy=49.02%
2019-05-23 09:45:41 iter:7189, accuracy=45.7%
2019-05-23 09:45:48 iter:7190, accuracy=56.05%
